In [1]:
import pandas as pd

data_file = pd.read_table('./data/Data.txt') # DataFrame 으로 불러오기
print(data_file)
print('총 샘플의 수 : ', len(data_file))

        label                                               text
0           5                                            배공빠르고 굿
1           2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2           5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3           2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4           5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...       ...                                                ...
199995      2                                    장마라그런가!!! 달지않아요
199996      5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997      5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998      5                                      넘이쁘고 쎄련되보이네요~
199999      5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다

[200000 rows x 2 columns]
총 샘플의 수 :  200000


In [2]:
data_file.drop_duplicates(subset=['text'], inplace=True) # 중복제거
data_file['text'] = data_file['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 기타 문자 제거
data_file = data_file.dropna(how = 'any') # null값 제거
print(data_file)
print('총 샘플의 수 : ', len(data_file))

data_file.to_csv('./data/processed_data.txt', sep = '\t', index = False)

        label                                               text
0           5                                            배공빠르고 굿
1           2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2           5  아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...
3           2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
4           5                   민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...       ...                                                ...
199995      2                                       장마라그런가 달지않아요
199996      5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997      5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998      5                                       넘이쁘고 쎄련되보이네요
199999      5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다

[199908 rows x 2 columns]
총 샘플의 수 :  199908


In [142]:
# data_loader.py
from torchtext.legacy import data

class DataLoader(object):
    def __init__(
        self, train_fn,
        batch_size = 64,
        valid_ratio=.2,
        device = -1,
        max_vocab = 999999,
        min_freq = 1,
        use_eos = False,
        shuffle = True
    ):
        super().__init__()
        
        self.label = data.Field(
            sequential = False,
            use_vocab = True,
            unk_token = None
        )
        self.text = data.Field(
            use_vocab = True,
            batch_first = True,
            include_lengths = False,
            eos_token = '<EOS>' if use_eos else None
        )

        train, valid = data.TabularDataset(
            path = train_fn,
            format = 'tsv',
            fields = [
                ('label', self.label),
                ('text', self.text)
            ]
        ).split(split_ratio = (1 - valid_ratio))
        
        self.train_loader, self.valid_loader = data.BucketIterator.splits(
            (train, valid),
            batch_size = batch_size,
            device = 'cuda:%d' % device if device >= 0 else 'cpu',
            shuffle = shuffle,
            sort_key = lambda x: len(x.text),
            sort_within_batch = True
        )
        
        self.label.build_vocab(train)
        self.text.build_vocab(train, max_size = max_vocab, min_freq = min_freq)

In [35]:
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self,
                 input_size,
                 word_vec_size,
                 hidden_size,
                 n_classes,
                 n_layers = 4,
                 dropout_p = .3
                ):
        self.vocab_size = input_size # vocabulary_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        self.rnn = nn.LSTM(input_size = word_vec_size,
                           hidden_size = hidden_size,
                           num_layers = n_layers,
                           dropout = dropout_p,
                           batch_first = True,
                           bidirectional = True
                          )
        self.generator = nn.Linear(hidden_size * 2, n_classes)
        # We use LogSoftmax + NLLLoss instead of Softmax + CrossEntropy
        self.activation = nn.LogSoftmax(dim = -1)
        
    def forward(self, x):
        # /x/ = (batch_size, length)
        x = self.emb(x)
        # /x/ = (batch_size, length, word_vec_dim)
        x, _ = self.rnn(x)
        # /x/ = (batch_size, length, hidden_size * 2)
        y = self.activation(self.generator(x[:, -1]))
        # /y/ = (batch_size, n_classes)
            
        return y

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

In [62]:
def read_text(fn):
    with open(fn, 'r') as f:
        lines = f.readlines()

        labels, texts = [], []
        for line in lines:
            if line.strip() != '':
                # The file should have tab delimited two columns.
                # First column indicates label field,
                # and second column indicates text field.
                label, text = line.strip().split('\t')
                labels += [label]
                texts += [text]

    return labels, texts


def get_grad_norm(parameters, norm_type=2):
    parameters = list(filter(lambda p: p.grad is not None, parameters))

    total_norm = 0

    try:
        for p in parameters:
            total_norm += (p.grad.data**norm_type).sum()
        total_norm = total_norm ** (1. / norm_type)
    except Exception as e:
        print(e)

    return total_norm


def get_parameter_norm(parameters, norm_type=2):
    total_norm = 0

    try:
        for p in parameters:
            total_norm += (p.data**norm_type).sum()
        total_norm = total_norm ** (1. / norm_type)
    except Exception as e:
        print(e)

    return total_norm

In [53]:
from copy import deepcopy

from ignite.engine import Engine
from ignite.engine import Events
from ignite.metrics import RunningAverage
from ignite.contrib.handlers.tqdm_logger import ProgressBar

VERBOSE_SILENT = 0
VERBOSE_EPOCH_WISE = 1
VERBOSE_BATCH_WISE = 2


class MyEngine(Engine):

    def __init__(self, func, model, crit, optimizer, config):
        # Ignite Engine does not have objects in below lines.
        # Thus, we assign class variables to access these object, during the procedure.
        self.model = model
        self.crit = crit
        self.optimizer = optimizer
        self.config = config

        super().__init__(func) # Ignite Engine only needs function to run.

        self.best_loss = np.inf
        self.best_model = None

        self.device = next(model.parameters()).device

    @staticmethod
    def train(engine, mini_batch):
        # You have to reset the gradients of all model parameters
        # before to take another step in gradient descent.
        engine.model.train() # Because we assign model as class variable, we can easily access to it.
        engine.optimizer.zero_grad()

        x, y = mini_batch.text, mini_batch.label
        x, y = x.to(engine.device), y.to(engine.device)

        x = x[:, :engine.config.max_length]

        # Take feed-forward
        y_hat = engine.model(x)

        loss = engine.crit(y_hat, y)
        loss.backward()

        # Calculate accuracy only if 'y' is LongTensor,
        # which means that 'y' is one-hot representation.
        if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):
            accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))
        else:
            accuracy = 0

        p_norm = float(get_parameter_norm(engine.model.parameters()))
        g_norm = float(get_grad_norm(engine.model.parameters()))

        # Take a step of gradient descent.
        engine.optimizer.step()

        return {
            'loss': float(loss),
            'accuracy': float(accuracy),
            '|param|': p_norm,
            '|g_param|': g_norm,
        }

    @staticmethod
    def validate(engine, mini_batch):
        engine.model.eval()

        with torch.no_grad():
            x, y = mini_batch.text, mini_batch.label
            x, y = x.to(engine.device), y.to(engine.device)

            x = x[:, :engine.config.max_length]

            y_hat = engine.model(x)

            loss = engine.crit(y_hat, y)

            if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):
                accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))
            else:
                accuracy = 0

        return {
            'loss': float(loss),
            'accuracy': float(accuracy),
        }

    @staticmethod
    def attach(train_engine, validation_engine, verbose=VERBOSE_BATCH_WISE):
        # Attaching would be repaeted for serveral metrics.
        # Thus, we can reduce the repeated codes by using this function.
        def attach_running_average(engine, metric_name):
            RunningAverage(output_transform=lambda x: x[metric_name]).attach(
                engine,
                metric_name,
            )

        training_metric_names = ['loss', 'accuracy', '|param|', '|g_param|']

        for metric_name in training_metric_names:
            attach_running_average(train_engine, metric_name)

        # If the verbosity is set, progress bar would be shown for mini-batch iterations.
        # Without ignite, you can use tqdm to implement progress bar.
        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(train_engine, training_metric_names)

        # If the verbosity is set, statistics would be shown after each epoch.
        if verbose >= VERBOSE_EPOCH_WISE:
            @train_engine.on(Events.EPOCH_COMPLETED)
            def print_train_logs(engine):
                print('Epoch {} - |param|={:.2e} |g_param|={:.2e} loss={:.4e} accuracy={:.4f}'.format(
                    engine.state.epoch,
                    engine.state.metrics['|param|'],
                    engine.state.metrics['|g_param|'],
                    engine.state.metrics['loss'],
                    engine.state.metrics['accuracy'],
                ))

        validation_metric_names = ['loss', 'accuracy']
        
        for metric_name in validation_metric_names:
            attach_running_average(validation_engine, metric_name)

        # Do same things for validation engine.
        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(validation_engine, validation_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @validation_engine.on(Events.EPOCH_COMPLETED)
            def print_valid_logs(engine):
                print('Validation - loss={:.4e} accuracy={:.4f} best_loss={:.4e}'.format(
                    engine.state.metrics['loss'],
                    engine.state.metrics['accuracy'],
                    engine.best_loss,
                ))

    @staticmethod
    def check_best(engine):
        loss = float(engine.state.metrics['loss'])
        if loss <= engine.best_loss: # If current epoch returns lower validation loss,
            engine.best_loss = loss  # Update lowest validation loss.
            engine.best_model = deepcopy(engine.model.state_dict()) # Update best model weights.

    @staticmethod
    def save_model(engine, train_engine, config, **kwargs):
        torch.save(
            {
                'model': engine.best_model,
                'config': config,
                **kwargs
            }, config.model_fn
        )


class Trainer():

    def __init__(self, config):
        self.config = config

    def train(
        self,
        model, crit, optimizer,
        train_loader, valid_loader,
    ):
        train_engine = MyEngine(
            MyEngine.train,
            model, crit, optimizer, self.config
        )
        validation_engine = MyEngine(
            MyEngine.validate,
            model, crit, optimizer, self.config
        )

        MyEngine.attach(
            train_engine,
            validation_engine,
            verbose=self.config.verbose
        )

        def run_validation(engine, validation_engine, valid_loader):
            validation_engine.run(valid_loader, max_epochs=1)

        train_engine.add_event_handler(
            Events.EPOCH_COMPLETED, # event
            run_validation, # function
            validation_engine, valid_loader, # arguments
        )
        validation_engine.add_event_handler(
            Events.EPOCH_COMPLETED, # event
            MyEngine.check_best, # function
        )

        train_engine.run(
            train_loader,
            max_epochs=self.config.n_epochs,
        )

        model.load_state_dict(validation_engine.best_model)

        return model

In [54]:
class Trainer():
    def __init__(self, config):
        self.config = config

    def train(
        self,
        model, crit, optimizer,
        train_loader, valid_loader,
    ):
        train_engine = MyEngine(
            MyEngine.train,
            model, crit, optimizer, self.config
        )
        validation_engine = MyEngine(
            MyEngine.validate,
            model, crit, optimizer, self.config
        )

        MyEngine.attach(
            train_engine,
            validation_engine,
            verbose=self.config.verbose
        )

        def run_validation(engine, validation_engine, valid_loader):
            validation_engine.run(valid_loader, max_epochs=1)

        train_engine.add_event_handler(
            Events.EPOCH_COMPLETED, # event
            run_validation, # function
            validation_engine, valid_loader, # arguments
        )
        validation_engine.add_event_handler(
            Events.EPOCH_COMPLETED, # event
            MyEngine.check_best, # function
        )

        train_engine.run(
            train_loader,
            max_epochs=self.config.n_epochs,
        )

        model.load_state_dict(validation_engine.best_model)

        return model

In [145]:
class Config():
    def __init__(self):
        self.model_fn = "./models/model.pth"
        self.train_fn = "./data/processed_data.txt"
        
        self.gpu_id = "mps"
        self.verbose = 2
        
        self.min_vocab_freq = 5
        self.max_vocab_size = 999999
        
        self.batch_size = 256
        self.n_epochs = 10
        
        self.word_vec_size = 256
        self.dropout = .3
        
        self.max_length = 256
        
        self.rnn = True
        self.hidden_size = 512
        self.n_layers = 4
        
        self.cnn = False
        self.use_batch_norm = True
        self.window_sizes = [3, 4, 5]
        self.n_filters = [100, 100, 100]

config = Config()

In [78]:
loaders = DataLoader(
    train_fn=config.train_fn,
    batch_size=config.batch_size,
    min_freq=config.min_vocab_freq,
    max_vocab=config.max_vocab_size,
    device=config.gpu_id
)

In [57]:
print(
    '|train| =', len(loaders.train_loader.dataset),
    '|valid| =', len(loaders.valid_loader.dataset),
)

|train| = 159927 |valid| = 39982


In [58]:
vocab_size = len(loaders.text.vocab)
n_classes = len(loaders.label.vocab)
print('|vocab| =', vocab_size, '|classes| =', n_classes)

|vocab| = 26113 |classes| = 5


In [73]:
if config.rnn is False and config.cnn is False:
    raise Exception('You need to specify an architecture to train. (--rnn or --cnn)')

In [63]:
if config.rnn:
    # Declare model and loss.
    model = RNNClassifier(
        input_size=vocab_size,
        word_vec_size=config.word_vec_size,
        hidden_size=config.hidden_size,
        n_classes=n_classes,
        n_layers=config.n_layers,
        dropout_p=config.dropout,
    )
    optimizer = optim.Adam(model.parameters())
    crit = nn.NLLLoss()
    print(model)

    if config.gpu_id >= 0:
        model.cuda(config.gpu_id)
        crit.cuda(config.gpu_id)

    rnn_trainer = Trainer(config)
    rnn_model = rnn_trainer.train(
        model,
        crit,
        optimizer,
        loaders.train_loader,
        loaders.valid_loader
    )

RNNClassifier(
  (emb): Embedding(26113, 256)
  (rnn): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (generator): Linear(in_features=1024, out_features=5, bias=True)
  (activation): LogSoftmax(dim=-1)
)


Epoch 1 - |param|=2.59e+03 |g_param|=4.72e-01 loss=8.8598e-01 accuracy=0.6215


Validation - loss=8.6510e-01 accuracy=0.6107 best_loss=inf


Epoch 2 - |param|=2.59e+03 |g_param|=4.26e-01 loss=8.2865e-01 accuracy=0.6377


Validation - loss=8.3318e-01 accuracy=0.6296 best_loss=8.6510e-01


...

Validation - loss=1.4098e+00 accuracy=0.5948 best_loss=8.2444e-01


Epoch 9 - |param|=2.62e+03 |g_param|=6.43e-01 loss=3.3403e-01 accuracy=0.8633


Validation - loss=1.5375e+00 accuracy=0.5996 best_loss=8.2444e-01


Epoch 10 - |param|=2.63e+03 |g_param|=6.42e-01 loss=3.0087e-01 accuracy=0.8801


Validation - loss=1.7107e+00 accuracy=0.5950 best_loss=8.2444e-01


In [66]:
print(__name__ == '__main__')

True


In [146]:
torch.save({
    'rnn': rnn_model.state_dict() if config.rnn else None,
    'cnn': cnn_model.state_dict() if config.cnn else None,
    'config': config,
    'vocab': loaders.text.vocab,
    'classes': loaders.label.vocab,
}, config.model_fn)

In [81]:
import sys

In [138]:
def read_text(input_string, max_length=256):
    '''
    Read text from standard input for inference.
    '''
    lines = []

    for line in input_string: # sys.stdin:
        if line.strip() != '':
            lines += [line.strip().split(' ')[:max_length]]

    return lines

def define_field():
    '''
    To avoid use DataLoader class, just declare dummy fields. 
    With those fields, we can retore mapping table between words and indice.
    '''
    return (
        data.Field(
            use_vocab=True,
            batch_first=True,
            include_lengths=False,
        ),
        data.Field(
            sequential=False,
            use_vocab=True,
            unk_token=None,
        )
    )

In [140]:
class test_Config():
    def __init__(self):
        self.model_fn = "./models/model.pth"
        self.gpu_id = -1
        self.batch_size = 256
        self.top_k = 1
        self.max_length = 256
        
        self.drop_rnn = False
        self.drop_cnn = True

test_config = test_Config()

In [127]:
input_string = [
    "이건 쓰레기야",
    "최고에요! ㅎㅎ",
    "구매하길 잘 했네요",
    "실망이에요.. 금방 망가지네요",
    "다시는 구매 안합니다.",
    "또 주문할게요.",
    "포장도 제대로 안되어있고 정말 엉망이네요",
    "내돈 돌려줘",
    "사이즈가 안맞아요",
    "따듯하고 좋아요 세탁하면 어떨지 아직은 모르겠네요 세탁후 줄어듬만 없으면 100점이에요",
    "이정도면 무난히 만족합니다. 다른분들은 모르겠지만 딱 제가 찾던 폴라티네요.",
    "남들이 욕하길래 정말 궁금해서 사봤어요 우와 세상에ㅎㅎ 잠옷이 생겼네요 요즘 좀 우울했는에 웃음이 나오네요ㅋㅋ 옷감은 무지 얇고 목은 왜이리 긴지 난 목이기니까 하면서 샀는데 어느나라에 목이 긴 부족에게 추천하고 싶네요 돈많고 좀 우울한 분에게 기분전환용으로 추천합니다 덕분에 웃었어요",
    "다신 안삼",
    "인연끊고 싶은 사람에게 추천해라",
    "이런 입지도 못하는 옷을 팔다니 싸더라도 입을 수 있게는 만들어야지 기장이 짧아서 배꼽이 보일지경ㅜㅜ",
    "한마디로 잘라 말하면 사지 마세요."
]

In [141]:
# saved_data = torch.load(
#     config.model_fn,
#     map_location='cpu' if config.gpu_id < 0 else 'cuda:%d' % config.gpu_id
# )

# train_config = saved_data['config']
# rnn_best = saved_data['rnn']
# cnn_best = saved_data['cnn']
# vocab = saved_data['vocab']
# classes = saved_data['classes']


train_config = config
rnn_best = rnn_model.state_dict() if config.rnn else None
cnn_best = cnn_model.state_dict() if config.cnn else None
vocab = loaders.text.vocab
classes = loaders.label.vocab

vocab_size = len(vocab)
n_classes = len(classes)

text_field, label_field = define_field()
text_field.vocab = vocab
label_field.vocab = classes

lines = read_text(input_string, max_length=test_config.max_length)
print(lines)

with torch.no_grad():
    ensemble = []
    if rnn_best is not None and not test_config.drop_rnn:
        # Declare model and load pre-trained weights.
        model = RNNClassifier(
            input_size=vocab_size,
            word_vec_size=train_config.word_vec_size,
            hidden_size=train_config.hidden_size,
            n_classes=n_classes,
            n_layers=train_config.n_layers,
            dropout_p=train_config.dropout,
        )
        model.load_state_dict(rnn_best)
        ensemble += [model]
    if cnn_best is not None and not test_config.drop_cnn:
        # Declare model and load pre-trained weights.
        model = CNNClassifier(
            input_size=vocab_size,
            word_vec_size=train_config.word_vec_size,
            n_classes=n_classes,
            use_batch_norm=train_config.use_batch_norm,
            dropout_p=train_config.dropout,
            window_sizes=train_config.window_sizes,
            n_filters=train_config.n_filters,
        )
        model.load_state_dict(cnn_best)
        ensemble += [model]

    y_hats = []
    # Get prediction with iteration on ensemble.
    for model in ensemble:
        if test_config.gpu_id >= 0:
            model.cuda(test_config.gpu_id)
        # Don't forget turn-on evaluation mode.
        model.eval()

        y_hat = []
        for idx in range(0, len(lines), test_config.batch_size):                
            # Converts string to list of index.
            x = text_field.numericalize(
                text_field.pad(lines[idx:idx + test_config.batch_size]),
                device='cuda:%d' % test_config.gpu_id if test_config.gpu_id >= 0 else 'cpu',
            )

            y_hat += [model(x).cpu()]
        # Concatenate the mini-batch wise result
        y_hat = torch.cat(y_hat, dim=0)
        # |y_hat| = (len(lines), n_classes)

        y_hats += [y_hat]

        model.cpu()
    # Merge to one tensor for ensemble result and make probability from log-prob.
    y_hats = torch.stack(y_hats).exp()
    # |y_hats| = (len(ensemble), len(lines), n_classes)
    y_hats = y_hats.sum(dim=0) / len(ensemble) # Get average
    # |y_hats| = (len(lines), n_classes)

    probs, indice = y_hats.topk(test_config.top_k)

    for i in range(len(lines)):
        sys.stdout.write('%s\t%s\n' % (
            ' '.join([classes.itos[indice[i][j]] for j in range(test_config.top_k)]), 
            ' '.join(lines[i])
        ))

[['이건', '쓰레기야'], ['최고에요!', 'ㅎㅎ'], ['구매하길', '잘', '했네요'], ['실망이에요..', '금방', '망가지네요'], ['다시는', '구매', '안합니다.'], ['또', '주문할게요.'], ['포장도', '제대로', '안되어있고', '정말', '엉망이네요'], ['내돈', '돌려줘'], ['사이즈가', '안맞아요'], ['따듯하고', '좋아요', '세탁하면', '어떨지', '아직은', '모르겠네요', '세탁후', '줄어듬만', '없으면', '100점이에요'], ['이정도면', '무난히', '만족합니다.', '다른분들은', '모르겠지만', '딱', '제가', '찾던', '폴라티네요.'], ['남들이', '욕하길래', '정말', '궁금해서', '사봤어요', '우와', '세상에ㅎㅎ', '잠옷이', '생겼네요', '요즘', '좀', '우울했는에', '웃음이', '나오네요ㅋㅋ', '옷감은', '무지', '얇고', '목은', '왜이리', '긴지', '난', '목이기니까', '하면서', '샀는데', '어느나라에', '목이', '긴', '부족에게', '추천하고', '싶네요', '돈많고', '좀', '우울한', '분에게', '기분전환용으로', '추천합니다', '덕분에', '웃었어요'], ['다신', '안삼'], ['인연끊고', '싶은', '사람에게', '추천해라'], ['이런', '입지도', '못하는', '옷을', '팔다니', '싸더라도', '입을', '수', '있게는', '만들어야지', '기장이', '짧아서', '배꼽이', '보일지경ㅜㅜ'], ['한마디로', '잘라', '말하면', '사지', '마세요.']]
2	이건 쓰레기야
5	최고에요! ㅎㅎ
5	구매하길 잘 했네요
2	실망이에요.. 금방 망가지네요
1	다시는 구매 안합니다.
5	또 주문할게요.
1	포장도 제대로 안되어있고 정말 엉망이네요
1	내돈 돌려줘
2	사이즈가 안맞아요
5	따듯하고 좋아요 세탁하면 어떨지 아직은 모르겠네요 세탁후 줄어듬만 없으면 100점이에요
2	이정도면 무난히 만족